# Convolutional Neural Networks (CNN) for Burn Area Classification with Sentinel-2 Imagery

## Import Library Packages

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import rasterio
import earthpy.plot as ep
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import from_levels_and_colors
import tensorflow as tf
from tensorflow.keras import layers, models
from IPython.display import display, Markdown
import seaborn as sns
from sklearn.utils import resample


In [ ]:
# Parameters
FEATURES = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12', 'dNBR', 'NDVI', 'NDWI', 'Burn_Label']
FOLDER_PATH = 'Raster_Train'

# Find all .tif files in the folder
tif_files = glob.glob(os.path.join(FOLDER_PATH, '*.tif'))

In [ ]:
# Load and concatenate all data
all_data = []

for file in tif_files:
    with rasterio.open(file) as src:
        bands = src.read()  # (bands, height, width)
        bands_reshaped = bands.reshape(bands.shape[0], -1).T  # (pixels, bands)
        df = pd.DataFrame(bands_reshaped, columns=FEATURES)
        all_data.append(df)

# Combine into a single DataFrame
df_all = pd.concat(all_data, ignore_index=True)

# Drop NaNs or invalid values (optional, but often needed)
df_all = df_all.dropna()

# Downsample the data by Burn_Label
df_majority = df_all[df_all['Burn_Label'] == 0]
df_minority = df_all[df_all['Burn_Label'] == 1]

df_majority_downsampled = resample(
    df_majority,
    replace=False,      # sample without replacement
    n_samples=len(df_minority),  # match minority class
    random_state=42
)

# Combine balanced data
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Shuffle the result
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Show preview
print(df_balanced.head())
print(df_balanced['Burn_Label'].value_counts())

## Split Data into Train and Test

In [ ]:
# Separate features and label
X = df_balanced[FEATURES[:-1]].values  # all bands except the last one
y = df_balanced[FEATURES[-1]].values   # last band is the label

# Remove pixels with no data if necessary (e.g., NaNs or a mask value)
valid_idx = ~np.isnan(X).any(axis=1)
X = X[valid_idx]
y = y[valid_idx]

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape for CNN: (samples, height, width, channels)
# We'll reshape each pixel as a 1x1 image with multiple bands
X_cnn = X_scaled.reshape(-1, 1, 1, X.shape[1])

# Binary classification assumed here (e.g., burned vs not burned)
y_cnn = (y > 0).astype(np.uint8)  # or apply thresholding logic

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_cnn, y_cnn, test_size=0.2, random_state=42)

print("Train Dataset Shape:", X_train.shape)
print("Test Dataset Shape:", X_test.shape)

## CNN Model Implementation

In [ ]:
# Define a simple CNN model
model = models.Sequential([
    layers.Input(shape=(1, 1, X.shape[1])),
    layers.Conv2D(16, (1, 1), activation='relu'),
    layers.Conv2D(32, (1, 1), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=256,
                    validation_data=(X_test, y_test))

## Evaluate the Model

### Plot Accuracy and Loss Graphs

In [ ]:
# Plot training & validation accuracy and loss
plt.figure(figsize=(12, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

### Classification Report and Confusion Matrix

In [ ]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(np.uint8)

# Classification report
print("\nGenerating classification report and confusion matrix...")
report = classification_report(y_test, y_pred, output_dict=True)
cm = confusion_matrix(y_test, y_pred)

# Create a summary of the results
cnn_result = [{
    'Classifier': 'Convolutional Neural Network',
    'Model Definition': model,
    'Class 0 - Precision': report['0']['precision'],
    'Class 0 - Recall': report['0']['recall'],
    'Class 0 - F1-Score': report['0']['f1-score'],
    'Class 1 - Precision': report['1']['precision'],
    'Class 1 - Recall': report['1']['recall'],
    'Class 1 - F1-Score': report['1']['f1-score'],
    'Average - Precision': report['macro avg']['precision'],
    'Average - Recall': report['macro avg']['recall'],
    'Average - F1-Score': report['macro avg']['f1-score'],
    'Accuracy': report['accuracy'],
    'Confusion Matrix': cm
}]
    
cnn_result_df = pd.DataFrame(cnn_result)
    
display(Markdown("### Classification Report of Convolutional Neural Network"))
display(cnn_result_df)

# Plot Confusion Matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix - Convolutional Neural Network (CNN)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()